Vaibhav Uniyal

22070126126

AIML B2

In [ ]:
pip install torch torchvision medmnist tensorboard scipy


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.models.inception import inception_v3
from medmnist import PathMNIST
from torch.utils.tensorboard import SummaryWriter
from scipy.linalg import sqrtm

In [ ]:
# Initialize TensorBoard
writer = SummaryWriter("runs/GAN_MedMNIST")

In [ ]:
# Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import os

# Manually creating the dataset directory
os.makedirs("/content/data", exist_ok=True)


In [ ]:
os.makedirs("generated", exist_ok=True)

In [ ]:
from medmnist import PathMNIST
from torchvision import transforms

# Define Transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load the Dataset
dataset = PathMNIST(root="/content/data", split="train", transform=transform, download=True)


100%|██████████| 206M/206M [04:57<00:00, 692kB/s]


In [ ]:
# Create DataLoader
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# Define Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(img.size(0), 1, 28, 28)

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1)
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        return self.model(img_flat)



In [ ]:
import torch

# Wasserstein GAN with Gradient Penalty
def compute_gradient_penalty(D, real_samples, fake_samples, device=None):
    if device is None:
        device = real_samples.device  # Infer device from input tensor

    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=device)
    interpolates = (alpha * real_samples + (1 - alpha) * fake_samples).requires_grad_(True)

    d_interpolates = D(interpolates)

    fake = torch.ones_like(d_interpolates, device=device)

    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True
    )[0]

    if gradients is None:
        raise RuntimeError("Gradients are None. Ensure D(interpolates) requires gradients.")

    gradients = gradients.view(gradients.size(0), -1)

    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()


In [ ]:
# Training Function
def train_gan(gan_type="LSGAN", epochs=50):
    generator = Generator().to(device)
    discriminator = Discriminator().to(device)

    if gan_type == "LSGAN":
        criterion = nn.MSELoss()
        optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    elif gan_type == "WGAN":
        optimizer_G = optim.RMSprop(generator.parameters(), lr=0.00005)
        optimizer_D = optim.RMSprop(discriminator.parameters(), lr=0.00005)

    elif gan_type == "WGAN-GP":
        optimizer_G = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.9))
        optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.9))

    for epoch in range(epochs):
        for i, (real_imgs, _) in enumerate(dataloader):
            real_imgs = real_imgs.to(device)

            # Train Discriminator
            optimizer_D.zero_grad()
            z = torch.randn(real_imgs.size(0), 100, device=device)
            fake_imgs = generator(z)

            real_validity = discriminator(real_imgs)
            fake_validity = discriminator(fake_imgs.detach())

            if gan_type == "LSGAN":
                real_loss = criterion(real_validity, torch.ones_like(real_validity))
                fake_loss = criterion(fake_validity, torch.zeros_like(fake_validity))
                d_loss = (real_loss + fake_loss) / 2
            elif gan_type == "WGAN":
                d_loss = -torch.mean(real_validity) + torch.mean(fake_validity)
                for p in discriminator.parameters():
                    p.data.clamp_(-0.01, 0.01)
            elif gan_type == "WGAN-GP":
                gradient_penalty = compute_gradient_penalty(discriminator, real_imgs, fake_imgs)
                d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + 10 * gradient_penalty

            d_loss.backward()
            optimizer_D.step()

            # Train Generator
            if i % 5 == 0:
                optimizer_G.zero_grad()
                fake_validity = discriminator(fake_imgs)
                g_loss = -torch.mean(fake_validity) if gan_type != "LSGAN" else criterion(fake_validity, torch.ones_like(fake_validity))
                g_loss.backward()
                optimizer_G.step()

        writer.add_scalar(f"{gan_type}/D_Loss", d_loss.item(), epoch)
        writer.add_scalar(f"{gan_type}/G_Loss", g_loss.item(), epoch)

        print(f"[{gan_type}] Epoch {epoch+1}/{epochs} - D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

        if epoch % 10 == 0:
            vutils.save_image(fake_imgs[:25], f"generated/{gan_type}_epoch_{epoch}.png", normalize=True)

    torch.save(generator.state_dict(), f"{gan_type}_generator.pth")
    print(f"{gan_type} Training Complete.")


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torch.utils.data import DataLoader

# Create directories for saving models
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the GAN training function
def train_gan(gan_type="LSGAN", epochs=50):
    print(f"\n🚀 Training {gan_type}...")

    # Initialize generator and discriminator
    generator = Generator().to(device)
    discriminator = Discriminator().to(device)

    if gan_type == "LSGAN":
        criterion = nn.MSELoss()
        optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    elif gan_type == "WGAN":
        optimizer_G = optim.RMSprop(generator.parameters(), lr=0.00005)
        optimizer_D = optim.RMSprop(discriminator.parameters(), lr=0.00005)

    elif gan_type == "WGAN-GP":
        optimizer_G = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.9))
        optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.9))

    else:
        raise ValueError(f"Unknown GAN type: {gan_type}")

    # Training loop
    for epoch in range(epochs):
        for i, (real_imgs, _) in enumerate(dataloader):
            real_imgs = real_imgs.to(device)
            optimizer_D.zero_grad()

            # Generate fake images
            z = torch.randn(real_imgs.size(0), 100, device=device)
            fake_imgs = generator(z)

            real_validity = discriminator(real_imgs)
            fake_validity = discriminator(fake_imgs.detach())

            if gan_type == "LSGAN":
                real_loss = criterion(real_validity, torch.ones_like(real_validity))
                fake_loss = criterion(fake_validity, torch.zeros_like(fake_validity))
                d_loss = (real_loss + fake_loss) / 2
            elif gan_type == "WGAN":
                d_loss = -torch.mean(real_validity) + torch.mean(fake_validity)
                for p in discriminator.parameters():
                    p.data.clamp_(-0.01, 0.01)
            elif gan_type == "WGAN-GP":
                gradient_penalty = compute_gradient_penalty(discriminator, real_imgs, fake_imgs)
                d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + 10 * gradient_penalty

            d_loss.backward()
            optimizer_D.step()

            # Train Generator
            if i % 5 == 0:
                optimizer_G.zero_grad()
                fake_validity = discriminator(fake_imgs)
                g_loss = -torch.mean(fake_validity) if gan_type != "LSGAN" else criterion(fake_validity, torch.ones_like(fake_validity))
                g_loss.backward()
                optimizer_G.step()

        print(f"[{gan_type}] Epoch {epoch+1}/{epochs} - D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    # ✅ Save the trained generator
    model_path = os.path.join(MODEL_DIR, f"{gan_type}_generator.pth")
    torch.save(generator.state_dict(), model_path)
    print(f"✅ Saved {gan_type} model to {model_path}")

    return generator  # Return the trained generator

# Train and save all GANs
gan_types = ["LSGAN", "WGAN", "WGAN-GP"]
trained_generators = {gan_type: train_gan(gan_type, epochs=50) for gan_type in gan_types}



🚀 Training LSGAN...
[LSGAN] Epoch 1/50 - D Loss: 0.1124, G Loss: 0.5152
[LSGAN] Epoch 2/50 - D Loss: 0.1429, G Loss: 0.8137
[LSGAN] Epoch 3/50 - D Loss: 0.0854, G Loss: 0.5972
[LSGAN] Epoch 4/50 - D Loss: 0.1222, G Loss: 0.7807
[LSGAN] Epoch 5/50 - D Loss: 0.1063, G Loss: 0.5290
[LSGAN] Epoch 6/50 - D Loss: 0.0928, G Loss: 0.5823
[LSGAN] Epoch 7/50 - D Loss: 0.1285, G Loss: 0.7184
[LSGAN] Epoch 8/50 - D Loss: 0.1102, G Loss: 0.7259
[LSGAN] Epoch 9/50 - D Loss: 0.1575, G Loss: 0.5468
[LSGAN] Epoch 10/50 - D Loss: 0.2292, G Loss: 0.4668
[LSGAN] Epoch 11/50 - D Loss: 0.1385, G Loss: 0.6293
[LSGAN] Epoch 12/50 - D Loss: 0.2288, G Loss: 0.5887
[LSGAN] Epoch 13/50 - D Loss: 0.1752, G Loss: 0.4263
[LSGAN] Epoch 14/50 - D Loss: 0.2065, G Loss: 0.3495
[LSGAN] Epoch 15/50 - D Loss: 0.1539, G Loss: 0.6565
[LSGAN] Epoch 16/50 - D Loss: 0.1401, G Loss: 0.4946
[LSGAN] Epoch 17/50 - D Loss: 0.1163, G Loss: 0.6041
[LSGAN] Epoch 18/50 - D Loss: 0.1436, G Loss: 0.5261
[LSGAN] Epoch 19/50 - D Loss: 0.19

In [ ]:
# Train all GANs
train_gan("WGAN-GP", epochs=50)
train_gan("LSGAN", epochs=50)
train_gan("WGAN", epochs=50)


[Epoch 1/50] [D loss: -7.4943] [G loss: 4.8444]
[Epoch 2/50] [D loss: -6.8537] [G loss: 6.2654]
[Epoch 3/50] [D loss: -5.6110] [G loss: 2.6428]
[Epoch 4/50] [D loss: -4.5170] [G loss: -0.0686]
[Epoch 5/50] [D loss: -5.2459] [G loss: 0.3764]
[Epoch 6/50] [D loss: -4.1785] [G loss: 0.8363]
[Epoch 7/50] [D loss: -4.4183] [G loss: 0.1295]
[Epoch 8/50] [D loss: -4.0026] [G loss: -0.2366]
[Epoch 9/50] [D loss: -3.4754] [G loss: 0.9665]
[Epoch 10/50] [D loss: -2.8785] [G loss: 1.9728]
[Epoch 11/50] [D loss: -2.4127] [G loss: 1.5790]
[Epoch 12/50] [D loss: -3.3286] [G loss: -0.2191]
[Epoch 13/50] [D loss: -3.1321] [G loss: 0.6837]
[Epoch 14/50] [D loss: -2.8933] [G loss: -0.3563]
[Epoch 15/50] [D loss: -2.4938] [G loss: -0.0013]
[Epoch 16/50] [D loss: -3.0848] [G loss: 1.1548]
[Epoch 17/50] [D loss: -2.0192] [G loss: -0.8589]
[Epoch 18/50] [D loss: -3.0897] [G loss: 1.0825]
[Epoch 19/50] [D loss: -2.6241] [G loss: -0.8277]
[Epoch 20/50] [D loss: -2.3579] [G loss: -2.4548]
[Epoch 21/50] [D loss

In [ ]:
#updated code for gans training saving the models and evaluationg the models
import os
import torch
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance

# Directory to save models and generated images
MODEL_DIR = "models"
IMAGE_DIR = "generated"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

# Generate images from a trained model
def generate_images(generator, num_samples=500):
    z = torch.randn(num_samples, 100, device=device)
    fake_imgs = generator(z)
    return fake_imgs

# Compute Inception Score
def compute_inception_score(fake_imgs):
    is_metric = InceptionScore(feature=2048).to(device)
    score = is_metric(fake_imgs)
    return score.item()

# Compute FID
def compute_fid(real_imgs, fake_imgs):
    fid_metric = FrechetInceptionDistance(feature=2048).to(device)

    fid_metric.update(real_imgs, real=True)
    fid_metric.update(fake_imgs, real=False)

    return fid_metric.compute().item()

# Visual Inspection
def visualize_images(fake_imgs, title="Generated Images", save_path=None):
    fake_imgs = fake_imgs[:16]  # Show 16 images
    grid = vutils.make_grid(fake_imgs, normalize=True).cpu().permute(1, 2, 0)

    plt.figure(figsize=(8, 8))
    plt.imshow(grid)
    plt.axis("off")
    plt.title(title)
    plt.show()

    if save_path:
        plt.savefig(save_path)
        print(f"Saved image: {save_path}")

# Train all GANs and store their models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gan_types = ["LSGAN", "WGAN", "WGAN-GP"]
trained_generators = {}

for gan_type in gan_types:
    model_path = os.path.join(MODEL_DIR, f"{gan_type}_generator.pth")

    if os.path.exists(model_path):
        print(f"\nSkipping training for {gan_type}, model already exists.")
        continue  # Skip training if model is already saved

    print(f"\nTraining {gan_type}...")
    generator = train_gan(gan_type, epochs=50)  # ✅ Train model
    trained_generators[gan_type] = generator  # ✅ Store trained generator

    # ✅ Save model to disk
    torch.save(generator.state_dict(), model_path)
    print(f"Saved {gan_type} model to {model_path}")

# Load trained models for evaluation
results = {}

for gan_type in gan_types:
    model_path = os.path.join(MODEL_DIR, f"{gan_type}_generator.pth")

    if not os.path.exists(model_path):
        print(f"Error: Model {model_path} not found! Skipping evaluation.")
        continue

    print(f"\nLoading trained {gan_type} model for evaluation...")

    generator = Generator().to(device)  # Initialize generator
    generator.load_state_dict(torch.load(model_path))  # Load saved model
    generator.eval()

    fake_images = generate_images(generator, num_samples=500)

    # Compute IS
    inception_score = compute_inception_score(fake_images)

    # Compute FID
    fid_score = compute_fid(real_images, fake_images)

    # Visual Inspection
    image_path = os.path.join(IMAGE_DIR, f"{gan_type}_generated.png")
    visualize_images(fake_images, title=f"{gan_type} Generated Images", save_path=image_path)

    # Store results
    results[gan_type] = {"Inception Score": inception_score, "FID Score": fid_score}

# Print final evaluation results
print("\n===== GAN Performance Comparison =====")
for gan, scores in results.items():
    print(f"{gan}: Inception Score = {scores['Inception Score']:.4f}, FID Score = {scores['FID Score']:.4f}")


In [ ]:
import os
MODEL_DIR = "models"
IMAGE_DIR = "generated"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gan_types = ["LSGAN", "WGAN", "WGAN-GP"]
trained_generators = {}
for gan_type in gan_types:
    model_path = os.path.join(MODEL_DIR, f"{gan_type}_generator.pth")

    if os.path.exists(model_path):
        print(f"\nSkipping training for {gan_type}, model already exists.")
        continue  # Skip training if model is already saved

    print(f"\nTraining {gan_type}...")
    generator = train_gan(gan_type, epochs=50)  # ✅ Train model
    trained_generators[gan_type] = generator  # ✅ Store trained generator

    # ✅ Save model to disk
    torch.save(generator.state_dict(), model_path)
    print(f"Saved {gan_type} model to {model_path}")


Training LSGAN...
[Epoch 1/50] [D loss: -7.3094] [G loss: 6.9294]
[Epoch 2/50] [D loss: -6.8139] [G loss: 5.7999]
[Epoch 3/50] [D loss: -7.1039] [G loss: 4.1957]
[Epoch 4/50] [D loss: -5.0366] [G loss: 1.6582]
[Epoch 5/50] [D loss: -4.3761] [G loss: 1.5712]
[Epoch 6/50] [D loss: -4.0551] [G loss: 0.7777]
[Epoch 7/50] [D loss: -3.5168] [G loss: -0.0991]
[Epoch 8/50] [D loss: -3.7128] [G loss: 0.7489]
[Epoch 9/50] [D loss: -2.9072] [G loss: 1.1476]
[Epoch 10/50] [D loss: -3.5268] [G loss: 0.1479]
[Epoch 11/50] [D loss: -3.3256] [G loss: 0.5671]
[Epoch 12/50] [D loss: -2.2581] [G loss: 0.6660]
[Epoch 13/50] [D loss: -2.8166] [G loss: 0.3038]
[Epoch 14/50] [D loss: -4.0532] [G loss: -0.6886]
[Epoch 15/50] [D loss: -2.5041] [G loss: -0.1326]
[Epoch 16/50] [D loss: -1.8693] [G loss: 1.4195]
[Epoch 17/50] [D loss: -2.6371] [G loss: 1.0665]
[Epoch 18/50] [D loss: -2.4649] [G loss: 1.4027]
[Epoch 19/50] [D loss: -1.5763] [G loss: -0.5785]
[Epoch 20/50] [D loss: -2.5599] [G loss: 0.0173]
[Epoch

AttributeError: 'NoneType' object has no attribute 'state_dict'

In [ ]:
import os
import torch
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance

# TensorBoard Setup
LOG_DIR = "runs/gan_evaluation"
writer = SummaryWriter(LOG_DIR)

# Generate images from a trained model
def generate_images(generator, num_samples=500):
    z = torch.randn(num_samples, 100, device=device)
    fake_imgs = generator(z)
    return fake_imgs

# Compute Inception Score
def compute_inception_score(fake_imgs):
    is_metric = InceptionScore(feature=2048).to(device)
    score = is_metric(fake_imgs)
    return score.item()

# Compute FID
def compute_fid(real_imgs, fake_imgs):
    fid_metric = FrechetInceptionDistance(feature=2048).to(device)

    fid_metric.update(real_imgs, real=True)
    fid_metric.update(fake_imgs, real=False)

    return fid_metric.compute().item()

# Log Images to TensorBoard
def log_images(fake_imgs, step, gan_type):
    fake_imgs = fake_imgs[:16]  # Select 16 images
    grid = vutils.make_grid(fake_imgs, normalize=True)

    writer.add_image(f"Generated Images/{gan_type}", grid, step)

results = {}

for step, gan_type in enumerate(gan_types):
    model_path = os.path.join(MODEL_DIR, f"{gan_type}_generator.pth")

    if not os.path.exists(model_path):
        print(f"Error: Model {model_path} not found! Skipping evaluation.")
        continue

    print(f"\nLoading trained {gan_type} model for evaluation...")

    generator = Generator().to(device)  # Initialize generator
    generator.load_state_dict(torch.load(model_path))  # Load saved model
    generator.eval()

    fake_images = generate_images(generator, num_samples=500)

    # Compute IS
    inception_score = compute_inception_score(fake_images)

    # Compute FID
    fid_score = compute_fid(real_images, fake_images)

    # Log Images to TensorBoard
    log_images(fake_images, step, gan_type)

    # Log Scores to TensorBoard
    writer.add_scalar(f"{gan_type}/Inception Score", inception_score, step)
    writer.add_scalar(f"{gan_type}/FID Score", fid_score, step)

    # Store results
    results[gan_type] = {"Inception Score": inception_score, "FID Score": fid_score}

# Print final evaluation results
print("\n===== GAN Performance Comparison =====")
for gan, scores in results.items():
    print(f"{gan}: Inception Score = {scores['Inception Score']:.4f}, FID Score = {scores['FID Score']:.4f}")

# Close TensorBoard Writer
writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs/gan_evaluation


In [ ]:
import os
import torch
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image.fid import FrechetInceptionDistance

# TensorBoard Setup
LOG_DIR = "runs/gan_evaluation"
writer = SummaryWriter(LOG_DIR)

# Directory to save models and generated images
MODEL_DIR = "models"
IMAGE_DIR = "generated"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR, exist_ok=True)

# Generate images from a trained model
def generate_images(generator, num_samples=500):
    z = torch.randn(num_samples, 100, device=device)
    fake_imgs = generator(z)
    return fake_imgs

# Compute Inception Score
def compute_inception_score(fake_imgs):
    is_metric = InceptionScore(feature=2048).to(device)
    score = is_metric(fake_imgs)
    return score.item()

# Compute FID
def compute_fid(real_imgs, fake_imgs):
    fid_metric = FrechetInceptionDistance(feature=2048).to(device)

    fid_metric.update(real_imgs, real=True)
    fid_metric.update(fake_imgs, real=False)

    return fid_metric.compute().item()

# Log Images to TensorBoard
def log_images(fake_imgs, step, gan_type):
    fake_imgs = fake_imgs[:16]  # Select 16 images
    grid = vutils.make_grid(fake_imgs, normalize=True)

    writer.add_image(f"Generated Images/{gan_type}", grid, step)

# Train all GANs and store their models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gan_types = ["LSGAN", "WGAN", "WGAN-GP"]
trained_generators = {}

for gan_type in gan_types:
    model_path = os.path.join(MODEL_DIR, f"{gan_type}_generator.pth")

    if os.path.exists(model_path):
        print(f"\nSkipping training for {gan_type}, model already exists.")
        continue  # Skip training if model is already saved

    print(f"\nTraining {gan_type}...")
    generator = train_gan(gan_type, epochs=50)  # ✅ Train model
    trained_generators[gan_type] = generator  # ✅ Store trained generator

    # ✅ Save model to disk
    torch.save(generator.state_dict(), model_path)
    print(f"Saved {gan_type} model to {model_path}")

# Load trained models for evaluation
results = {}

for step, gan_type in enumerate(gan_types):
    model_path = os.path.join(MODEL_DIR, f"{gan_type}_generator.pth")

    if not os.path.exists(model_path):
        print(f"Error: Model {model_path} not found! Skipping evaluation.")
        continue

    print(f"\nLoading trained {gan_type} model for evaluation...")

    generator = Generator().to(device)  # Initialize generator
    generator.load_state_dict(torch.load(model_path))  # Load saved model
    generator.eval()

    fake_images = generate_images(generator, num_samples=500)

    # Compute IS
    inception_score = compute_inception_score(fake_images)

    # Compute FID
    fid_score = compute_fid(real_images, fake_images)

    # Log Images to TensorBoard
    log_images(fake_images, step, gan_type)

    # Log Scores to TensorBoard
    writer.add_scalar(f"{gan_type}/Inception Score", inception_score, step)
    writer.add_scalar(f"{gan_type}/FID Score", fid_score, step)

    # Store results
    results[gan_type] = {"Inception Score": inception_score, "FID Score": fid_score}

# Print final evaluation results
print("\n===== GAN Performance Comparison =====")
for gan, scores in results.items():
    print(f"{gan}: Inception Score = {scores['Inception Score']:.4f}, FID Score = {scores['FID Score']:.4f}")

# Close TensorBoard Writer
writer.close()


In [ ]:
# Function to Display Generated Images
def show_generated_images():
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    for i, (gan_name, _) in enumerate(gan_models.items()):
        img = plt.imread(f"generated/{gan_name}_samples.png")
        axes[i].imshow(img)
        axes[i].axis("off")
        axes[i].set_title(gan_name)

    plt.show()


In [ ]:
# Show Sample Images from Each GAN
show_generated_images()